In [ ]:
#TASK 1 - Install matplotlib and deap
#       - Import all necessary libraries

In [ ]:
# TODO: Task 2 - Define the number of locations and vehicles

num_locations = # Define the number of locations (eg 10)

locations = # Create a list of tuples representing location coordinates -
            # try to use a random number generator
            # (x, y) could be between 0 and 100

depot = # Define the coordinates for the depot

num_vehicles = # Define the number of vehicles - (eg 3)

In [ ]:
# TASK 3 - Genetic Algorithm Setup
       # - Figure out how to represent an individual OR encode an individual

creator.create("FitnessMin", )
creator.create("Individual", )

#TASK 4 - Starting with the toolbox and tools

toolbox = base.Toolbox()
toolbox.register("indices", random.sample, range(num_locations), num_locations)
toolbox.register("individual", #finish defining how to build an individual)
toolbox.register("population", #finish defining how to build an population)

In [ ]:
# TASK 5 and 6

def evalVRP(individual):
    # TODO: Task 5 - Write the fitness evaluation function
    #              - Return (total distance, standard deviation)



#TASK 6 - Register the evaluate function in the toolbox

In [ ]:
!pip install deap

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from deap import base, creator, tools, algorithms
import random

In [ ]:
num_locations = 10
locations = [(np.random.uniform(0, 100), np.random.uniform(0, 100)) for _ in range(num_locations)]
depot = (0, 0)
num_vehicles = 3

In [ ]:
creator.create("FitnessMin", base.Fitness, weights = (-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMin)

In [ ]:
toolbox = base.Toolbox()
toolbox.register("indices", random.sample, range(num_locations), num_locations)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [ ]:
'''def evalVRP(individual):
      total_distance = np.linalg.norm(np.array(depot) - np.array(locations[individual[0]]))

      for i in range(len(individual) - 1):
          total_distance += np.linalg.norm(np.array(locations[individual[i]]) - np.array(locations[individual[i + 1]]))

      total_distance += np.linalg.norm(np.array(locations[individual[-1]]) - np.array(depot))

      return (total_distance,)

toolbox.register("evaluate", evalVRP)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)'''

In [ ]:
def evalVRP(individual):
    total_distance = 0
    distances = []  # Track distance traveled by each vehicle for balance calculation
    # Split the list of locations among vehicles, ensuring each starts and ends at the depot
    for i in range(num_vehicles):

        vehicle_route = [depot] + [locations[individual[j]] for j in range(i, len(individual), num_vehicles)] + [depot]

        # Calculate total distance traveled by this vehicle
        vehicle_distance = 0
        for k in range(len(vehicle_route) - 1):
            vehicle_distance += np.linalg.norm(np.array(vehicle_route[k]) - np.array(vehicle_route[k + 1]))

        total_distance += vehicle_distance
        distances.append(vehicle_distance)

    balance_penalty = np.std(distances)  # Use standard deviation of distances as a penalty for imbalance among vehicles
    return total_distance, balance_penalty

toolbox.register("evaluate", evalVRP)

* mate
* mutation
* selection






In [ ]:
#HINTS

#Select is simpler, try tournament selection or roulette selection. Look up how to add that.

#FOR mutation - mutShuffleIndexes

#For mate - cxPartialyMatched, cxOrdered

In [ ]:
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)

In [ ]:
def plot_routes(individual, locations, depot, num_vehicles, title="Routes"):
    plt.figure(figsize=(10, 8))
    plt.title(title)
    plt.scatter(*zip(*locations), color='blue', label='Locations')
    plt.scatter(*depot, color='red', marker='s', s=100, label='Depot')
    colors = plt.cm.jet(np.linspace(0, 1, num_vehicles))

    for i in range(num_vehicles):
        vehicle_route = [depot] + [locations[individual[j]] for j in range(i, len(individual), num_vehicles)] + [depot]
        x_coords, y_coords = zip(*vehicle_route)
        plt.plot(x_coords, y_coords, marker='o', linestyle='-', color=colors[i], label=f'Vehicle {i+1}')

    plt.xlabel("X Coordinate")
    plt.ylabel("Y Coordinate")
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
def main():
    random.seed(42)  # Seed for reproducibility
    pop = toolbox.population(n=300)  # Generate initial population
    hof = tools.HallOfFame(1)  # Hall of Fame to store the best individual

    # Setup statistics to track
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)

    # Run the genetic algorithm
    algorithms.eaSimple(pop, toolbox, 0.7, 0.2, 300, stats=stats, halloffame=hof)

    # Plot the best route found
    plot_routes(hof[0], locations, depot, num_vehicles, "Optimal Route")
    return pop, stats, hof

# --- Run the Algorithm and Print Results ---
if __name__ == "__main__":
    # Task 9 - Call the main function, run the algorithm, and print the results
    population, statistics, hall_of_fame = main()

    # Print some results
    best_individual = hall_of_fame[0]
    best_fitness = best_individual.fitness.values

    print("Best Individual:", best_individual)
    print("Best Fitness (Total Distance, Balance Penalty):", best_fitness)

    avg_fitness = statistics.compile(population)["avg"]
    min_fitness = statistics.compile(population)["min"]
    print(f"Average fitness: {avg_fitness}")
    print(f"Min fitness: {min_fitness}")